##### **Comparison of 2 tables from different semantic models**

In [1]:
import pandas as pd
import sempy.fabric as fabric
from sempy.fabric import FabricDataFrame
from sempy.dependencies import plot_dependency_metadata

StatementMeta(, b776186c-3274-4831-b3f2-24b946f362c8, 3, Finished, Available, Finished)

**'iREd' model measures**

In [3]:
fabric.list_measures(workspace="UDP Enterprise WS", dataset="iRED Explorer Model").head()

StatementMeta(, dedea49d-cf0e-4b76-80c4-f0c5d12f0300, 5, Finished, Available, Finished)

,Table Name,Measure Name,Measure Expression,Measure Data Type,Measure Hidden,Measure Display Folder,Measure Description,Format String,Data Category,Detail Rows Definition,Format String Definition
0,iRED Measures,Availability,\n(\n DIVIDE(\n DISTINCTCOUNT('iRED ...,Double,False,,% of stores with one or more facings of the me...,0.0%;-0.0%;0.0%,,None,None
1,iRED Measures,Sample Size,\nCALCULATE(\n DISTINCTCOUNT('iRED Meas...,Int64,False,,Total number of stores surveyed.\n,"#,0",,None,None
2,iRED Measures,Total # of Facings,"\nSUMX ( 'iRED Measures', 'iRED Measures'[faci...",Int64,False,,Total count of facings for measured products.\n,"#,0",,None,None
3,iRED Measures,Total # of SKUs,\nDISTINCTCOUNT('iRED Measures'[productid]),Int64,False,,Total distinct count of SKUs available in the ...,"#,0",,None,None
4,iRED Measures,Average # of SKUs,"\nDIVIDE ( [Total # of SKUs], [Sample Size] )",Double,False,,Average distinct count of SKUs available in th...,"#,0.0",,None,None


**'Share' model measures**

In [2]:
fabric.list_measures(workspace="UDP Enterprise WS", dataset="Share of Market by Customer").head()

StatementMeta(, dedea49d-cf0e-4b76-80c4-f0c5d12f0300, 4, Finished, Available, Finished)

,Table Name,Measure Name,Measure Expression,Measure Data Type,Measure Hidden,Measure Display Folder,Measure Description,Format String,Data Category,Detail Rows Definition,Format String Definition
0,Share of Market (Customer),Dollars,SUM('Share of Market (Customer)'[dollars_]),Double,False,Base Measures,,"\$#,0.00;(\$#,0.00);\$#,0.00",,None,None
1,Share of Market (Customer),Units,SUM('Share of Market (Customer)'[units_]),Double,False,Base Measures,,"#,0.00",,None,None
2,Share of Market (Customer),Any Display Dollars,SUM('Share of Market (Customer)'[any_display_d...,Double,False,Base Measures,,"\$#,0.00;(\$#,0.00);\$#,0.00",,None,None
3,Share of Market (Customer),Any Feature Dollars,SUM('Share of Market (Customer)'[any_feature_d...,Double,False,Base Measures,,"\$#,0.00;(\$#,0.00);\$#,0.00",,None,None
4,Share of Market (Customer),Any Promo Dollars,SUM('Share of Market (Customer)'[any_promo_dol...,Double,False,Base Measures,,"\$#,0.00;(\$#,0.00);\$#,0.00",,None,None


In [4]:
#Reading the tables
table = fabric.read_table("iRED Explorer Model", "Product")
table_share = fabric.read_table("Share of Market by Customer","Product")

StatementMeta(, a934ca7a-49d8-4f18-8d59-4355304619c1, 6, Finished, Available, Finished)

In [4]:
#Reading csv from lakehouse
table_iRed_Prod = spark.read.format("csv").option("header",True).load("Files/Product_iRed.csv")
table_share_Prod = spark.read.format("csv").option("header", True).load("Files/Product_share.csv")

StatementMeta(, dedea49d-cf0e-4b76-80c4-f0c5d12f0300, 6, Finished, Available, Finished)

In [4]:
#Schema's of both tables
schema1 = table_iRed_Prod.schema
schema2 = table_share_Prod.schema

StatementMeta(, 8303a61a-1566-4990-b0bf-9f279baca6c9, 6, Finished, Available, Finished)

In [8]:
from tabulate import tabulate

comparison_results = []

# Create sets of field names for both schemas
schema1_field_names = {field.name for field in schema1.fields}
schema2_field_names = {field.name for field in schema2.fields}

# Set of all field names in both schemas
all_field_names = schema1_field_names.union(schema2_field_names)

# Iterate over all field names and compare
for field_name in all_field_names:
    in_schema1 = field_name in schema1_field_names
    in_schema2 = field_name in schema2_field_names

    # Determine match status
    if in_schema1 and in_schema2:
        match_status = "Match"
    else:
        match_status = "Mismatch"

    # Get field names for comparison results
    field1_name = field_name if in_schema1 else "Not Present"
    field2_name = field_name if in_schema2 else "Not Present"

    # Append the comparison result
    comparison_results.append([field1_name, field2_name, match_status])

# Define headers for the table
headers = ["Schema1 Field Name", "Schema2 Field Name", "Match Status"]

# Print the comparison results in tabular form
print(tabulate(comparison_results, headers=headers, tablefmt="grid"))


StatementMeta(, 8303a61a-1566-4990-b0bf-9f279baca6c9, 10, Finished, Available, Finished)

+---------------------------+--------------------------------+----------------+
| Schema1 Field Name        | Schema2 Field Name             | Match Status   |
+===========================+================================+================+
| ko_consumption            | Not Present                    | Mismatch       |
+---------------------------+--------------------------------+----------------+
| Not Present               | Sparkling vs. Non-Sparkling    | Mismatch       |
+---------------------------+--------------------------------+----------------+
| Not Present               | Product Key                    | Mismatch       |
+---------------------------+--------------------------------+----------------+
| Package                   | Package                        | Match          |
+---------------------------+--------------------------------+----------------+
| SSD/Still                 | Not Present                    | Mismatch       |
+---------------------------+-----------

In [7]:
from tabulate import tabulate

# List to store results
comparison_results = []

# Create dictionaries to map field names to their nullability for both schemas
schema1_fields = {field.name: field.nullable for field in schema1.fields}
schema2_fields = {field.name: field.nullable for field in schema2.fields}

# Set of all field names in both schemas
all_field_names = set(schema1_fields.keys()).union(schema2_fields.keys())

# Iterate over all field names and compare nullability
for field_name in all_field_names:
    nullable1 = schema1_fields.get(field_name, "Not Present")
    nullable2 = schema2_fields.get(field_name, "Not Present")

    # Compare nullability
    if nullable1 == nullable2:
        match_status = "Identical"
    else:
        match_status = "Mismatch"

    # Get field names for comparison results
    field1_name = field_name if field_name in schema1_fields else "Not Present"
    field2_name = field_name if field_name in schema2_fields else "Not Present"

    # Append the comparison result
    comparison_results.append([field1_name, field2_name, nullable1, nullable2, match_status])

# Define headers for the table
headers = ["Schema1 Field Name", "Schema2 Field Name", "Schema1 Nullable", "Schema2 Nullable", "Match Status"]

# Print the comparison results in tabular form
print(tabulate(comparison_results, headers=headers, tablefmt="grid"))


StatementMeta(, 8303a61a-1566-4990-b0bf-9f279baca6c9, 9, Finished, Available, Finished)

+---------------------------+--------------------------------+--------------------+--------------------+----------------+
| Schema1 Field Name        | Schema2 Field Name             | Schema1 Nullable   | Schema2 Nullable   | Match Status   |
+===========================+================================+====================+====================+================+
| ko_consumption            | Not Present                    | True               | Not Present        | Mismatch       |
+---------------------------+--------------------------------+--------------------+--------------------+----------------+
| Not Present               | Product Key                    | Not Present        | True               | Mismatch       |
+---------------------------+--------------------------------+--------------------+--------------------+----------------+
| Package                   | Package                        | True               | True               | Identical      |
+-----------------------

In [5]:
from tabulate import tabulate

# List to store results
comparison_results = []

# Create dictionaries to map field names to their data types for both schemas
schema1_fields = {field.name: field.dataType for field in schema1.fields}
schema2_fields = {field.name: field.dataType for field in schema2.fields}

# Set of all field names in both schemas
all_field_names = set(schema1_fields.keys()).union(schema2_fields.keys())

# Iterate over all field names and compare data types
for field_name in all_field_names:
    data_type1 = schema1_fields.get(field_name, "Not Present")
    data_type2 = schema2_fields.get(field_name, "Not Present")

    # Compare field data types
    if data_type1 == data_type2:
        match_status = "Identical"
    else:
        match_status = "Mismatch"

    # Get field names for comparison results
    field1_name = field_name if field_name in schema1_fields else "Not Present"
    field2_name = field_name if field_name in schema2_fields else "Not Present"

    # Append the comparison result
    comparison_results.append([field1_name, field2_name, match_status, data_type1, data_type2])

# Define headers for the table
headers = ["Schema1 Field Name", "Schema2 Field Name", "Match Status", "Schema1 DataType", "Schema2 DataType"]

# Print the comparison results in tabular form
print(tabulate(comparison_results, headers=headers, tablefmt="grid"))


StatementMeta(, 8303a61a-1566-4990-b0bf-9f279baca6c9, 7, Finished, Available, Finished)

+---------------------------+--------------------------------+----------------+--------------------+--------------------+
| Schema1 Field Name        | Schema2 Field Name             | Match Status   | Schema1 DataType   | Schema2 DataType   |
+===========================+================================+================+====================+====================+
| ko_consumption            | Not Present                    | Mismatch       | StringType()       | Not Present        |
+---------------------------+--------------------------------+----------------+--------------------+--------------------+
| Not Present               | Product Key                    | Mismatch       | Not Present        | StringType()       |
+---------------------------+--------------------------------+----------------+--------------------+--------------------+
| Package                   | Package                        | Identical      | StringType()       | StringType()       |
+-----------------------

In [2]:
fabric.list_datasets()

StatementMeta(, d0a334fc-c5c9-4778-b107-76fc406f1c9e, 4, Finished, Available, Finished)

,Dataset Name,Dataset ID,Created Timestamp,Last Update
0,fabric_purview_poc,c12761f4-1808-4b20-9e63-b4cfbd9935d3,2021-02-12 23:00:58,NaT
1,Sample_lakehouse_765,4c6ea573-cd8d-4d6a-9cfb-1ed4a0da8b71,2021-02-12 23:00:58,NaT
2,POC Purview,ac8ee863-ace8-40b5-954c-7ec9f96efd75,2021-02-12 23:00:58,NaT
3,factory_calendar_x08_cleansed,81761111-5d33-48d7-84f1-d1354c625b23,2024-05-29 09:57:35,NaT
4,factory_calendar_x08_raw,159fa32f-2ff4-463e-8dd2-908cfef3f0e3,2024-05-29 09:57:35,NaT
5,Sample Semantic model,780e9851-ed63-4905-a55c-c1fed8df4589,2024-06-01 14:58:00,NaT
6,Sample Dash,e15efe8c-b41f-4afd-8f64-11bbd01e30f6,2024-06-03 22:52:57,NaT
7,Outlet Query Tool,0e783473-8a0b-4190-8d20-94f7f879ae06,2023-06-22 20:50:50,NaT
8,jaylake,d55ca035-fab4-4944-8931-c4bcddf22973,2021-02-12 23:00:58,NaT
9,iRED Explorer Model,0334b804-bad1-4572-babd-697accb7ec65,2019-09-17 05:50:29,NaT


In [44]:
# Dataset 1
fabric.list_measures(dataset = 'Share of Market by Customer')

StatementMeta(, 86f5035b-6b8c-4213-bee6-5b1f0d51fad6, 46, Finished, Available)

,Table Name,Measure Name,Measure Expression,Measure Data Type,Measure Hidden,Measure Display Folder,Measure Description,Format String,Data Category,Detail Rows Definition,Format String Definition
0,Share of Market (Customer),Dollars,SUM('Share of Market (Customer)'[dollars_]),Double,False,Base Measures,,"\$#,0.00;(\$#,0.00);\$#,0.00",,None,None
1,Share of Market (Customer),Units,SUM('Share of Market (Customer)'[units_]),Double,False,Base Measures,,"#,0.00",,None,None
2,Share of Market (Customer),Any Display Dollars,SUM('Share of Market (Customer)'[any_display_d...,Double,False,Base Measures,,"\$#,0.00;(\$#,0.00);\$#,0.00",,None,None
3,Share of Market (Customer),Any Feature Dollars,SUM('Share of Market (Customer)'[any_feature_d...,Double,False,Base Measures,,"\$#,0.00;(\$#,0.00);\$#,0.00",,None,None
4,Share of Market (Customer),Any Promo Dollars,SUM('Share of Market (Customer)'[any_promo_dol...,Double,False,Base Measures,,"\$#,0.00;(\$#,0.00);\$#,0.00",,None,None
...,...,...,...,...,...,...,...,...,...,...,...
60,Share of Market (Customer),Share of Units MTD vs. PY (% pts),\n//get the latest month\nVAR mtd_cy = max('Ca...,Double,False,Share of Units,The difference in 'Percentage Points' between ...,0.0,,None,None
61,Share of Market (Customer),Share of Units R13W,\n//this varible will define the number of wee...,Double,False,Share of Units,Share over a rolling 13 week period,0.0%;-0.0%;0.0%,,None,None
62,Share of Market (Customer),Share of Units R13W vs. PY (% pts),\n//this varible will define the number of wee...,Double,False,Share of Units,The difference in 'Percentage Points' between ...,0.0,,None,None
63,Share of Market (Customer),Share of Units R52W,\n//this varible will define the number of wee...,Double,False,Share of Units,Share over a rolling 52 week period,0.0%;-0.0%;0.0%,,None,None


In [3]:

#Dax query on Dataset 1

df_dax = fabric.evaluate_dax(
    "Share of Market by Customer",
    """
    EVALUATE
    SUMMARIZECOLUMNS(
    'Product'[Brand],
    'Customer'[Channel],
    "Any Promo Dollars", [Any Promo Dollars])
    """)

df_dax.head()


StatementMeta(, d0a334fc-c5c9-4778-b107-76fc406f1c9e, 5, Finished, Available, Finished)

,Product[Brand],Customer[Channel],[Any Promo Dollars]
0,RC,Walmart,11313940.611
1,SMARTWATER ANTIOXIDANT,Walmart,5548.121
2,PRIVATE LABEL,Walmart,1224200805.512
3,CASCADE ICE,Walmart,0.0
4,FLORIDA'S NATURAL,Walmart,10554765.911


In [3]:
# Reading Dataset 2 with Same schema but different dataset and table names
fabric.list_tables(dataset='Share of Market second')

StatementMeta(, 95a4c1b1-2f04-4fea-b133-57ff31465724, 5, Finished, Available, Finished)

,Name,Description,Hidden,Data Category,Type
0,Share of Market 2,rms_mdl vw_fact_customerweekly,False,,Table
1,Product_2,rms_mdl vw_Dim_Category,False,,Table
2,Customer_2,rms_mdl vw_Dim_Customers,False,,Table
3,Calendar_2,rms_mdl vw_Dim_Period,False,Time,Table


In [4]:
#dax query on dataset 2 with different table names
df_dax = fabric.evaluate_dax(
    "Share of Market second",
    """
    EVALUATE
    SUMMARIZECOLUMNS(
    'Product_2'[Brand],
    'Customer_2'[Channel],
    "Dollars", [Dollars])
    """)

df_dax.head()

StatementMeta(, 95a4c1b1-2f04-4fea-b133-57ff31465724, 6, Finished, Available, Finished)

,Product_2[Brand],Customer_2[Channel],[Dollars]
0,RC,Walmart,33744819.93
1,SMARTWATER ANTIOXIDANT,Walmart,36441.08
2,PRIVATE LABEL,Walmart,9340395694.320326
3,CASCADE ICE,Walmart,99.57
4,FLORIDA'S NATURAL,Walmart,164636425.34


###### **Experiment**

In [12]:
fabric.list_measures(dataset = 'test2_schema')

StatementMeta(, d0a334fc-c5c9-4778-b107-76fc406f1c9e, 14, Finished, Available, Finished)

,Table Name,Measure Name,Measure Expression,Measure Data Type,Measure Hidden,Measure Display Folder,Measure Description,Format String,Data Category,Detail Rows Definition,Format String Definition
0,Sales,Total Sales,SUM(Sales[SalesAmount]),Int64,False,,,0,,None,None
1,Sales,Total Quantity,SUM(Sales[Quantity]),Int64,False,,,0,,None,None
2,Sales,Average Sales Amount,AVERAGE(Sales[SalesAmount]),Double,False,,,,,None,None
3,Sales,Total Sales by Product,"\nSUMX(\n SUMMARIZE(\n 'Product',\n ...",Int64,False,,,0,,None,None
4,Sales,Sales of Specific Region,\nSUMX (\n FILTER (\n VALUES ('Custo...,Int64,False,,,0,,None,None


In [13]:
fabric.list_measures(dataset = 'test1_schema')

StatementMeta(, d0a334fc-c5c9-4778-b107-76fc406f1c9e, 15, Finished, Available, Finished)

,Table Name,Measure Name,Measure Expression,Measure Data Type,Measure Hidden,Measure Display Folder,Measure Description,Format String,Data Category,Detail Rows Definition,Format String Definition
0,Sales,Total Sales,SUM(Sales[SalesAmount]),Int64,False,,,0,,None,None
1,Sales,Total Quantity,SUM(Sales[Quantity]),Int64,False,,,0,,None,None
2,Sales,Average Sales Amount,AVERAGE(Sales[SalesAmount]),Double,False,,,,,None,None
3,Sales,Sales Summary,,Int64,False,,,,,None,None


In [11]:
# DAX query
dax_query = """
EVALUATE
SUMMARIZECOLUMNS (
    'Customer'[Region],
    "Total Sales", SUM('Sales'[SalesAmount])
)
"""

# Execute the DAX query
df_dax = fabric.evaluate_dax(
    dataset = 'test1_schema',
    dax_string = dax_query
)

# result
print("Total Sales by Region")
print(df_dax)

StatementMeta(, d0a334fc-c5c9-4778-b107-76fc406f1c9e, 13, Finished, Available, Finished)

Total Sales by Region
  Customer[Region]  [Total Sales]
0            North            200
1             East            150
2            South            300


In [18]:
# Define the corrected DAX query
dax_query = """
EVALUATE
VAR _AvgDisplaysFiltered =
IF (
    ISFILTERED('Store Area'[Location]) && VALUES('Store Area'[Location]) = "Display",
    DIVIDE (
        CALCULATE (
            DISTINCTCOUNT ( 'iRED Measures'[scene_id] ),
            FILTER('Store Area', 'Store Area'[Location] = "Display")
        ),
        [Sample Size]
    ),
    0
)
VAR _AvgDisplays =
DIVIDE (
    CALCULATE (
        DISTINCTCOUNT ( 'iRED Measures'[scene_id] ),
        FILTER('Store Area', 'Store Area'[Location] = "Display")
    ),
    [Sample Size]
)
VAR _Results =
SWITCH(
    TRUE(),
    HASONEFILTER('Store Area'[Location]), _AvgDisplaysFiltered,
    _AvgDisplays
)
RETURN
SELECTCOLUMNS(
    ROW("Description", "Average # of Displays", "Result", _Results),
    "Description", "Result"
)
"""

StatementMeta(, 24389882-25a0-459c-b93f-2e92879b3355, 20, Finished, Available, Finished)

In [19]:
# Evaluate the DAX query using fabric.evaluate_dax
df = fabric.evaluate_dax(
    dataset = 'iRED Explorer Model',
    dax_string = dax_query
)

StatementMeta(, 24389882-25a0-459c-b93f-2e92879b3355, 21, Finished, Available, Finished)

###### **Metrics**

In [11]:
from pyspark.sql.functions import col

# Group by the 'package' column and count the occurrences
distinct_counts = table_iRed_Prod.groupBy("package").count()

# distinct counts
distinct_counts.show()

StatementMeta(, a934ca7a-49d8-4f18-8d59-4355304619c1, 13, Finished, Available, Finished)

+--------------------+-----+
|             package|count|
+--------------------+-----+
|                  1L|   56|
|2OZ SINGLE BOTTLE...|   26|
|                96OZ|   57|
|     .5L 35PK BOTTLE|   20|
|                59OZ|   38|
|            16OZ SNG|    2|
|        6-6.8OZ 10PK|   81|
|     .5L 40PK BOTTLE|   18|
|  21OZ SINGLE BOTTLE|   41|
|     20OZ 8PK BOTTLE|   51|
|         24OZ SINGLE|   95|
|17OZ - 19.9OZ 6PK...|   12|
|     .5L 12PK BOTTLE|  122|
|  24OZ SINGLE BOTTLE|    6|
| 320OZ SINGLE BOTTLE|    1|
|         8OZ 8PK CAN|    1|
|           13OZ 12PK|    1|
|     18OZ SINGLE CAN|    2|
|      .5L 6PK BOTTLE|  290|
|           9.5OZ SNG|   12|
+--------------------+-----+
only showing top 20 rows



In [20]:
from pyspark.sql.functions import col

# Group by the 'package' column and count the occurrences
distinct_counts = table_iRed_Prod.groupBy("CBR Category").count()

# distinct counts
distinct_counts.show()

StatementMeta(, a934ca7a-49d8-4f18-8d59-4355304619c1, 22, Finished, Available, Finished)

+--------------------+-----+
|        CBR Category|count|
+--------------------+-----+
|     SPARKLING WATER| 2278|
|       RTD SH COFFEE|  595|
|       JUICE SH BTLR| 1034|
|RTD COMPLETE NUTR...|  540|
|FLAT FLAVOR NON I...|  167|
|FLAT UNFLAVORED P...|  343|
|          RTD CH TEA|  218|
|              SSD IC| 1191|
|FLAT FLAVOR INGRE...|  399|
|          RTD SH TEA| 1057|
|CH RTD COCONUT WATER|   14|
|        SPORTS DRINK|  678|
|              SSD FC| 3008|
|         JUICE SH WH| 2981|
|     TRADITIONAL NHB|  531|
|    JUICE CHILLED WH|  723|
|          BASE WATER|  562|
|            KOMBUCHA|  334|
|        JUICE FROZEN|   16|
|       ENERGY DRINKS| 1173|
+--------------------+-----+
only showing top 20 rows



In [27]:
from pyspark.sql.functions import col

# Group by the 'package' column and count the occurrences
distinct_counts = table_iRed_Prod.groupBy("Flavor").count()

# distinct counts
distinct_counts.show()

StatementMeta(, a934ca7a-49d8-4f18-8d59-4355304619c1, 29, Finished, Available, Finished)

+--------------------+-----+
|              Flavor|count|
+--------------------+-----+
|   ALL OTHER FLAVORS| 1501|
|               MANGO|  249|
|         CHERRY COLA|   81|
|            ESPRESSO|   42|
|           STARLIGHT|    6|
|           RASPBERRY|  269|
|REMAINING ONE-FLA...|   42|
|              ALMOND|    1|
|               PUNCH|   16|
|         ORANGE COLA|    2|
|             PUMPKIN|   13|
|    SUBTLY SWEET TEA|    6|
|   FRUIT/FRUIT PUNCH|  362|
|             unknown| 4743|
|           GREEN TEA|    2|
|           MARGARITA|   46|
|         COFFEE COLA|    8|
|          BIRCH BEER|   16|
|               PEACH|  402|
|         GREEN APPLE|   12|
+--------------------+-----+
only showing top 20 rows



In [7]:
# Define the dataset name
dataset = "iRED Explorer Model"

# Define the DAX query string
dax_query = """
EVALUATE
VAR TotalSKUs = [Total # of SKUs]
VAR SampleSize = [Sample Size]
RETURN
SUMMARIZECOLUMNS(
    "Average # of SKUs", DIVIDE(TotalSKUs, SampleSize)
)
"""

# Execute the DAX query using fabric.evaluate_dax
result_df = fabric.evaluate_dax(dataset = dataset, dax_string = dax_query)

# Display the result
print(result_df)


StatementMeta(, dedea49d-cf0e-4b76-80c4-f0c5d12f0300, 9, Finished, Available, Finished)

   [Average # of SKUs]
0             0.061467


In [15]:
dax_query = """
EVALUATE
ROW("Total # of SKUs", DISTINCTCOUNT('Share of Market'[productid]))
"""
# Execute the DAX query using fabric.evaluate_dax
result_df = fabric.evaluate_dax(dataset = 'Share of Market by Customer', dax_string = dax_query)

# Display the result
print(result_df)

StatementMeta(, dedea49d-cf0e-4b76-80c4-f0c5d12f0300, 17, Finished, Available, Finished)

AdomdErrorResponseException: Query (3, 38) Cannot find table '<ccon>Share of Market</ccon>'.

Technical Details:
RootActivityId: 8a404349-4a7c-4b7f-9f2c-2cf363fbc1ed
Date (UTC): 6/25/2024 7:44:59 PM
   at Microsoft.AnalysisServices.AdomdClient.XmlaClient.CheckForSoapFault(XmlReader reader, XmlaResult xmlaResult, Boolean throwIfError)
   at Microsoft.AnalysisServices.AdomdClient.XmlaClient.CheckForError(XmlReader reader, XmlaResult xmlaResult, Boolean throwIfError)
   at Microsoft.AnalysisServices.AdomdClient.XmlaClient.SendMessage(Boolean endReceivalIfException, Boolean readSession, Boolean readNamespaceCompatibility)
   at Microsoft.AnalysisServices.AdomdClient.XmlaClient.ExecuteStatement(String statement, IDictionary connectionProperties, IDictionary commandProperties, IDataParameterCollection parameters, Boolean isMdx)
   at Microsoft.AnalysisServices.AdomdClient.AdomdConnection.XmlaClientProvider.Microsoft.AnalysisServices.AdomdClient.IExecuteProvider.ExecuteTabular(CommandBeha